In [82]:
import cv2
import numpy as np
import getopt
import sys
import random

In [83]:
ratio=0.85
min_match=10
smoothing_window_size=800
estimation_thresh = 0.60

In [84]:
#
# Runs sift algorithm to find features
#
def findFeatures(img):
    print("Finding Features...")
    sift=cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)

    img = cv2.drawKeypoints(img, keypoints, img)
    cv2.imwrite('sift_keypoints.png', img)

    return keypoints, descriptors

In [85]:
#
# Matches features given a list of keypoints, descriptors, and images
#
def matchFeatures(kp1, kp2, desc1, desc2, img1, img2):
    print("Matching Features...")    
    matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(desc1, desc2, k=2)
    
    return matches

In [86]:
#
# Computers a homography from 4-correspondences
#
def calculateHomography(correspondences):
    #loop through correspondences and create assemble matrix
    aList = []
    for corr in correspondences:
        p1 = np.matrix([corr.item(0), corr.item(1), 1])
        p2 = np.matrix([corr.item(2), corr.item(3), 1])

        a1 = [p1.item(0), p1.item(1), 1, 0, 0, 0, -p2.item(0)*p1.item(0), -p2.item(0)*p1.item(1), -p2.item(0)]
        a2 = [0, 0, 0, p1.item(0), p1.item(1), 1, -p2.item(1)*p1.item(0), -p2.item(1)*p1.item(1), -p2.item(1)]
        
        aList.append(a1)
        aList.append(a2)

    matrixA = np.matrix(aList)

    #svd composition
    u, s, v = np.linalg.svd(matrixA)

    #reshape the min singular value into a 3 by 3 matrix
    h = np.reshape(v[8], (3, 3))

    #normalize and now we have h
    h = (1/h.item(8)) * h
    return h

In [87]:
#
#Calculate the geometric distance between estimated points and original points
#
def geometricDistance(correspondence, h):

    p1 = np.transpose(np.matrix([correspondence[0].item(0), correspondence[0].item(1), 1]))
    estimatep2 = np.dot(h, p1)
    estimatep2 = (1/estimatep2.item(2))*estimatep2

    p2 = np.transpose(np.matrix([correspondence[0].item(2), correspondence[0].item(3), 1]))
    error = p2 - estimatep2
    return np.linalg.norm(error)

In [88]:
#
#Runs through ransac algorithm, creating homographies from random correspondences
#
def ransac(corr, thresh):
    maxInliers = []
    finalH = None
    for i in range(1000):
        #find 4 random points to calculate a homography
        corr1 = corr[random.randrange(0, len(corr))]
        corr2 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((corr1, corr2))
        corr3 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr3))
        corr4 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr4))

        #call the homography function on those points
        h = calculateHomography(randomFour)
        inliers = []
        error_list = []

        for i in range(len(corr)):
            d = geometricDistance(corr[i], h)
            error_list.append(d)
            if d < 5:
                inliers.append(corr[i])

        if len(inliers) > len(maxInliers):
            maxInliers = inliers
            finalH = h
        print("Corr size: ", len(corr), " NumInliers: ", len(inliers), "Max inliers: ", len(maxInliers))

        if len(maxInliers) > (len(corr)*thresh):
            break

    return finalH, maxInliers, error_list

In [89]:
def create_mask(img1,img2,version):
    height_img1 = img1.shape[0]
    width_img1 = img1.shape[1]
    width_img2 = img2.shape[1]
    height_panorama = height_img1
    width_panorama = width_img1 +width_img2
    offset = int(smoothing_window_size / 2)
    barrier = img1.shape[1] - int(smoothing_window_size / 2)
    mask = np.zeros((height_panorama, width_panorama))
    if version== 'left_image':
        mask[:, barrier - offset:barrier + offset ] = np.tile(np.linspace(1, 0, 2 * offset ).T, (height_panorama, 1))
        mask[:, :barrier - offset] = 1
    else:
        mask[:, barrier - offset :barrier + offset ] = np.tile(np.linspace(0, 1, 2 * offset ).T, (height_panorama, 1))
        mask[:, barrier + offset:] = 1
    return cv2.merge([mask, mask, mask])

In [90]:
def blending(img1,img2):
    ### 추가 ###
    H = finalH
    #H = registration(img1,img2)

    print(f'H:{H}')
    height_img1 = img1.shape[0]
    width_img1 = img1.shape[1]
    width_img2 = img2.shape[1]
    height_panorama = height_img1
    width_panorama = width_img1 +width_img2

    panorama1 = np.zeros((height_panorama, width_panorama, 3))
    mask1 = create_mask(img1,img2,version='left_image')
    panorama1[0:img1.shape[0], 0:img1.shape[1], :] = img1
    panorama1 *= mask1
    mask2 = create_mask(img1,img2,version='right_image')
    panorama2 = cv2.warpPerspective(img2, H, (width_panorama, height_panorama))*mask2
    panorama3 = cv2.warpPerspective(img2, H, (width_panorama, height_panorama))
    result=panorama1+panorama2

    rows, cols = np.where(result[:, :, 0] != 0)
    min_row, max_row = min(rows), max(rows) + 1
    min_col, max_col = min(cols), max(cols) + 1
    final_result = result[min_row:max_row, min_col:max_col, :]
    return final_result, panorama1, panorama2, panorama3

In [91]:
#
# Main parses argument list and runs the functions
#

#query image
img1 = cv2.imread('./img/img_31.jpg', 0)
#train image
img2 = cv2.imread('./img/img_32.jpg', 0)

#find features and keypoints
correspondenceList = []
if img1 is not None and img2 is not None:
    kp1, desc1 = findFeatures(img1)
    kp2, desc2 = findFeatures(img2)
    print("Found keypoints in " + 'img1' + ": " + str(len(kp1)))
    print("Found keypoints in " + 'img2' + ": " + str(len(kp2)))
    keypoints = [kp1,kp2]
    matches = matchFeatures(kp1, kp2, desc1, desc2, img1, img2)
    
    ### 추가 ###
    good_points = []
    good_matches=[]
    for m1, m2 in matches:
        if m1.distance < ratio * m2.distance:
            good_points.append((m1.trainIdx, m1.queryIdx))
            good_matches.append([m1])
    
    img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches, None, flags=2)
    cv2.imwrite('matching.jpg', img3)
    
    if len(good_points) > min_match:       
        for (i, j) in good_points:
            (x1, y1) = kp1[j].pt
            (x2, y2) = kp2[i].pt
            correspondenceList.append([x2, y2, x1, y1])

    corrs = np.matrix(correspondenceList)

    #run ransac algorithm
    finalH, inliers, error_list = ransac(corrs, estimation_thresh)
    print("Final homography: ", finalH)
    print("Final inliers count: ", len(inliers))

    f = open('homography.txt', 'w')
    f.write("Final homography: \n" + str(finalH)+"\n")
    f.write("Final inliers count: " + str(len(inliers)))
    f.close()
else:
    print("Failed to load images")
    # print(f'error: {error_list}')

###여기까지 추가

Finding Features...
Finding Features...
Found keypoints in img1: 13806
Found keypoints in img2: 12089
Matching Features...
Corr size:  2792  NumInliers:  126 Max inliers:  126
Corr size:  2792  NumInliers:  1273 Max inliers:  1273
Corr size:  2792  NumInliers:  1423 Max inliers:  1423
Corr size:  2792  NumInliers:  88 Max inliers:  1423
Corr size:  2792  NumInliers:  2116 Max inliers:  2116
Final homography:  [[ 5.07189829e-01 -4.82657045e-02  5.27734329e+02]
 [-1.20198636e-01  8.28147688e-01  5.81084774e+01]
 [-3.65804403e-04  2.59249826e-06  1.00000000e+00]]
Final inliers count:  2116


In [92]:
img1 = cv2.imread('./img/img_31.jpg')
img2 = cv2.imread('./img/img_32.jpg')

In [93]:
final, pano1, pano2, pano3 = blending(img1, img2)

cv2.imwrite('pano1.jpg', pano1)
cv2.imwrite('pano2.jpg', pano2)
cv2.imwrite('pano3.jpg', pano3)
cv2.imwrite('panorama.jpg', final)

H:[[ 5.07189829e-01 -4.82657045e-02  5.27734329e+02]
 [-1.20198636e-01  8.28147688e-01  5.81084774e+01]
 [-3.65804403e-04  2.59249826e-06  1.00000000e+00]]


True